<a href="https://colab.research.google.com/github/JFernandez696/Projects_v00/blob/main/BestFunctionAll_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import math as m

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import optimize 
from scipy import interpolate
from scipy  import stats

from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.stattools import durbin_watson

from sklearn.linear_model import LinearRegression

import xlrd 
import os
import tqdm
import inspect

import warnings
warnings.simplefilter("ignore")

In [2]:
# KPSS & ADF test
def kpss_test(series, **kw): 
    print ('Results of Kwiatkowski-Phillips-Schmidt-Shin Test:')   
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    kys, vls =[], []
    for key, value in critical_values.items():
        kys.append(key), vls.append(value)
    print('Critial Values (CV) :')
    df = pd.DataFrame({'(%ST at)':kys, 'CV':vls})
    print(df)
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

def adf_test(series):
    print ('Results of Augmented Dickey-Fuller Test:')
    statistic, p_value, n_lags, number_observations_used, critical_values, icbest = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    kys, vls =[], []
    for key, value in critical_values.items():
        kys.append(key), vls.append(value)
    print('Critial Values (CV) :')
    df = pd.DataFrame({'(%ST at)':kys, 'CV':vls})
    print(df)
    print(f'Result: The series is {"not " if p_value > 0.05 else ""}stationary')

In [3]:
# Series de Tiempo
def get_rolling_mean(values, window, min_periods=0):
    """Return rolling mean of given values, using specified window size."""
    return values.rolling(window=window,min_periods=min_periods).mean()

def get_rolling_std(values, window, min_periods=0):
    """Return rolling standard deviation of given values, using specified window size."""
    # TODO: Compute and return rolling standard deviation
    return values.rolling(window=window,min_periods=min_periods).std()

def get_bollinger_bands(rm, rstd):
    """Return upper and lower Bollinger Bands."""
    # TODO: Compute upper_band and lower_band
    upper_band = rstd * 2 + rm
    lower_band =  rm - rstd * 2
    return upper_band, lower_band

In [4]:
# Graph and Test series de Tiempo
def test_stationarity(timeseries,xlabel=None,ylabel=None,ystd='std'):
    # Determining rolling statistics
    rolmean = timeseries.rolling(60,min_periods=0).mean()
    rolstd  = timeseries.rolling(60,min_periods=0).std()
    # Plot rolling statistics:
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    ax.plot(timeseries, color='blue',label='Original')
    ax.plot(rolmean, color='red', label='Rolling Mean')
    ax2 = ax.twinx()
    ax2.plot(rolstd, color='black', alpha=0.75,label = 'Rolling Std')
    ax2.legend(loc=0)
    ax.legend(loc=0)
    ax.grid(color='black', linestyle='--', linewidth=0.5, which='both', axis='both', alpha=0.15)
    ax2.grid(color='black', linestyle='--', linewidth=0.5, which='both', axis='both', alpha=0.15)
    plt.title('Rolling Mean & Standard Deviation')
    ax.set_xlabel(xlabel),ax.set_ylabel(ylabel),ax2.set_ylabel(ystd)
    plt.show()
    print('-------------------------------------------------')
    adf_test(timeseries)
    print('-------------------------------------------------')
    kpss_test(timeseries)

In [5]:
# Fluid Properties
def EOS(P, T , SGg=0.65): 
  # P : Pressure ,psia
  # T : Temperature, °F
  # SGg : Specific Gravity Gas, dec
  Ppc = 677 + 15*SGg - 37.5*(SGg**2)
  Tpc = 168 + 325*SGg - 12.5*(SGg**2)  
  Tpr = (T+459.9)/Tpc
  Ppr = P/Ppc
  A = 1.39*((Tpr - 0.92)**(0.5)) - 0.36*Tpr - 0.1
  B = (0.62 - 0.23*Tpr)*Ppr + ((0.066/(Tpr - 0.86)) - 0.037)*Ppr**2 + 0.32*(Ppr**6)/(10**(9*(Tpr - 1)))
  C = 0.132 - 0.32*m.log10(Tpr)
  D = 10**(0.3106 - 0.49*Tpr + 0.1824*Tpr**2)
  z = A + (1 - A)/m.exp(B) + C*Ppr**D 
  return z

def PolyGasPower(PolyHead, Qg, rhoInlet, ηPoly):
  HPCal = PolyHead*(Qg*694.444)*rhoInlet/33000/(ηPoly/100)
  return HPCal

def PdCompFinal(Qg, Ps, Ts, NS, SGg, CR, n, MW, R, COT, IPD, ηPoly, ηMech):
  # Stage 1
  Ps1 = Ps
  Ts1 = Ts
  zs1     = EOS(Ps1,Ts1,SGg=0.65)     # Stage 1 Suction   compressibility fact (z)
  rhos1   = (Ps1*MW)/(R*(Ts1+459.9)) 

  Pd1 = Ps1*CR
  Td1 = (Ts1+459.9)*(Pd1/Ps1)**((n-1)/n)-460
  zd1     = EOS(Pd1,Ts1, SGg=0.65)      # Stage 1 Discharge compressibility fact (z)
  rhod1   = (Pd1*MW)/(R*(Td1+459.9))

  za1        = (zs1 +zd1)/2
  PolyHead1  = Ps1*za1*n/(rhos1*(n-1))*((Pd1/Ps1)**((n-1)/n)-1)*144   # Polytropic head, ft-lbf/lbm
  PolyGasHP1 = PolyGasPower(PolyHead1, Qg, rhos1, ηPoly)              # Polytropic gas power, hp

  # Stage 2
  Ps2 = Pd1-IPD 
  Ts2 = COT
  zs2     = EOS(Ps2,Ts2, SGg=0.65)     # Stage 2 Suction   compressibility fact (z)
  rhos2   = (Ps2*MW)/(R*(Ts2+459.9)) 

  Pd2 = Ps2*CR 
  Td2 = (Ts2+459.9)*(Pd2/Ps2)**((n-1)/n)-460
  zd2     = EOS(Pd2,Td2, SGg=0.65)      # Stage 2 Discharge compressibility fact (z)
  rhod2   = (Pd2*MW)/(R*(Td2+459.9))

  za2        = (zs2 +zd2)/2
  PolyHead2  = Ps2*za2*n/(rhos2*(n-1))*((Pd2/Ps2)**((n-1)/n)-1)*144   # Polytropic head, ft-lbf/lbm
  PolyGasHP2 = PolyGasPower(PolyHead2, Qg, rhos1, ηPoly)              # Polytropic gas power, hp

  # Stage 3
  Ps3 = Pd2-IPD 
  Ts3 = COT
  zs3     = EOS(Ps3,Ts3, SGg=0.65)     # Stage 3 Suction   compressibility fact (z)
  rhos3   = (Ps3*MW)/(R*(Ts3+459.9)) 

  Pd3 = Ps3*CR 
  Td3 = (Ts3+459.9)*(Pd3/Ps3)**((n-1)/n)-460
  zd3     = EOS(Pd3,Td3, SGg=0.65)      # Stage 2 Discharge compressibility fact (z)
  rhod3   = (Pd3*MW)/(R*(Td3+459.9))

  za3        = (zs3 +zd3)/2
  PolyHead3  = Ps3*za3*n/(rhos3*(n-1))*((Pd3/Ps3)**((n-1)/n)-1)*144   # Polytropic head, ft-lbf/lbm
  PolyGasHP3 = PolyGasPower(PolyHead3, Qg, rhos1, ηPoly)              # Polytropic gas power, hp

  # Stage 4
  Ps4 = Pd3-IPD 
  Ts4 = COT
  zs4     = EOS(Ps4,Ts4, SGg=0.65)     # Stage 4 Suction   compressibility fact (z)
  rhos4   = (Ps4*MW)/(R*(Ts4+459.9)) 

  Pd4 = Ps4*CR 
  Td4 = (Ts4+459.9)*(Pd4/Ps4)**((n-1)/n)-460
  zd4     = EOS(Pd4,Td4, SGg=0.65)      # Stage 4 Discharge compressibility fact (z)
  rhod4   = (Pd4*MW)/(R*(Td4+459.9))

  za4        = (zs4 +zd4)/2
  PolyHead4  = Ps4*za4*n/(rhos4*(n-1))*((Pd4/Ps4)**((n-1)/n)-1)*144   # Polytropic head, ft-lbf/lbm
  PolyGasHP4 = PolyGasPower(PolyHead4, Qg, rhos1, ηPoly)              # Polytropic gas power, hp
  if NS==1:
    Pdf = Pd1
    TGP = PolyGasHP1
    LGP = [PolyGasHP1]
  elif NS==2:
    Pdf = Pd2
    TGP = PolyGasHP1+ PolyGasHP2
    LGP = [PolyGasHP1,PolyGasHP2]
  elif NS==3:
    Pdf = Pd3
    TGP = PolyGasHP1 + PolyGasHP2 + PolyGasHP3
    LGP = [PolyGasHP1,PolyGasHP2,PolyGasHP3]
  elif NS==4:
    Pdf = Pd4
    TGP = PolyGasHP1 + PolyGasHP2 + PolyGasHP3 + PolyGasHP4
    LGP  = [PolyGasHP1,PolyGasHP2,PolyGasHP3,PolyGasHP4]
  else:
    "Check Number Satage"  
  TGP = TGP            # Total Gas Power, hps
  SP  = TGP*100/ηMech  # BrakePower (shaft power)
  ML  = SP-TGP         # Add for mechanical losses
  return Pdf, LGP, TGP, SP, ML

def FuntionOCT(Psbh, PI, WHP, API, fw, GLR, IDt, Dpt, Dov, ang, Vat0=1000, FB=0.07):
  """
  Function Optimum cycle time calculation
  Psbh : Reservoir Pressure, psi
  PI   : Productivity Index, bpd/psi
  WHP  : Tubing Head Pressure, psi
  API  : Oil Density, °API
  fw   : Water Cut , dec
  GLR  : Gas Liquid Ratio, scf/STB
  IDt  : Inside Diameter, in
  Dpt  : Reservoir Depth - MD , ft
  Dov  : Operating Valve Depth - MD , ft
  ang  : Average angle at Dov
  Vat0 : The liquid slug velocity in Mft/min (initially assumed equal 1000 ft/min)
  FB   : Fall Back tubing  it assumed aprox 0.05, 0.0
  """  
  DENST = (167 - 1.43*API)/1000  # Approximate true pressure gradient in psi/ft
  fg    = (1 + Dov/54000)**1.524
  A     = Psbh - (Dpt - Dov)*DENST*m.cos(ang*m.pi/180) - WHP * fg

  Bt    = 0.97143*IDt**2
  Densf = 0.433*(fw + (1 - fw)*(141.5/(API + 131.5)))
  Gamma = Densf*PI/(1.44*Bt)
  C4    = m.exp(Gamma*Dov/Vat0)
  CM    = FB*(Dov/1000)
  C2    = 1 - CM

  def func(T):
    return (m.exp(Gamma*T) - C4)*(m.exp(Gamma*T) - CM*C4)/(Gamma*m.exp(Gamma*T)*C2*C4)-T
  TC   = optimize.newton(func, 3*Dov/Vat0)
  Tiny = Dov/Vat0
  Tf   = TC - Tiny            # The liquid column regeneration time, Tf
  Qini = A*(m.exp(Gamma*Tf) - 1)/(1000*Densf*(m.exp(Gamma*Tf) - CM))
  Pto  = 1000*Qini*Densf + WHP*fg
  Qf   = (Qini/m.cos(ang*m.pi/180))*Bt*(1 - FB*Dov/ 1000)*1440/TC
  Qg   = Qf*GLR*1E-6
  return [TC, Qf, Pto, Qg, WHP, Tiny]

def seleccionar_distribuciones(familia='realall', verbose=True):
    '''
    Esta función selecciona un subconjunto de las distribuciones disponibles
    en scipy.stats
    
    Parameters
    ----------
    familia : {'realall', 'realline', 'realplus', 'real0to1', 'discreta'}
        realall: distribuciones de la familia `realline` + `realplus`
        realline: distribuciones continuas en el dominio (-inf, +inf)
        realplus: distribuciones continuas en el dominio [0, +inf)
        real0to1: distribuciones continuas en el dominio [0,1]
        discreta: distribuciones discretas
        
    verbose : bool
        Si se muestra información de las distribuciones seleccionadas
        (the default `True`).
        
    Returns
    -------
    distribuciones: list
        listado con las distribuciones (los objetos) seleccionados.
        
    Raises
    ------
    Exception
        Si `familia` es distinto de 'realall', 'realline', 'realplus', 'real0to1',
        o 'discreta'.
        
    Notes
    -----
        Las distribuciones levy_stable y vonmises han sido excluidas por el momento.

    '''
    
    distribuciones = [getattr(stats,d) for d in dir(stats) \
                     if isinstance(getattr(stats,d), (stats.rv_continuous, stats.rv_discrete))]
    
    exclusiones = ['levy_stable', 'vonmises']
    distribuciones = [dist for dist in distribuciones if dist.name not in exclusiones]
            
    dominios = {
        'realall' : [-np.inf, np.inf],
        'realline': [np.inf,np.inf],
        'realplus': [0, np.inf],
        'real0to1': [0, 1], 
        'discreta': [None, None],
    }

    distribucion = []
    tipo = []
    dominio_inf = []
    dominio_sup = []

    for dist in distribuciones:
        distribucion.append(dist.name)
        tipo.append(np.where(isinstance(dist, stats.rv_continuous), 'continua', 'discreta'))
        dominio_inf.append(dist.a)
        dominio_sup.append(dist.b)
    
    info_distribuciones = pd.DataFrame({
                            'distribucion': distribucion,
                            'tipo': tipo,
                            'dominio_inf': dominio_inf,
                            'dominio_sup': dominio_sup
                          })

    info_distribuciones = info_distribuciones \
                          .sort_values(by=['dominio_inf', 'dominio_sup'])\
                          .reset_index(drop=True)
    
    if familia in ['realall', 'realline', 'realplus', 'real0to1']:
        info_distribuciones = info_distribuciones[info_distribuciones['tipo']=='continua']
        condicion = (info_distribuciones['dominio_inf'] == dominios[familia][0]) & \
                    (info_distribuciones['dominio_sup'] == dominios[familia][1]) 
        info_distribuciones = info_distribuciones[condicion].reset_index(drop=True)
        
    if familia in ['discreta']:
        info_distribuciones = info_distribuciones[info_distribuciones['tipo']=='discreta']
        
    seleccion = [dist for dist in distribuciones \
                 if dist.name in info_distribuciones['distribucion'].values]
    
    
    if verbose:
        print("---------------------------------------------------")
        print("       Distribuciones seleccionadas                ")
        print("---------------------------------------------------")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
            print(info_distribuciones)
    
    return seleccion

def comparar_distribuciones(x, familia='realall', ordenar='aic', verbose=True):
    '''
    Esta función selecciona y ajusta un subconjunto de las distribuciones 
    disponibles en scipy.stats. Para cada distribución calcula los valores de
    Log Likelihood, AIC y BIC.
    
    Parameters
    ----------
    x : array_like
        datos con los que ajustar la distribución.
        
    familia : {'realall', 'realline', 'realplus', 'real0to1', 'discreta'}
        realall: distribuciones de la familia `realline` + `realplus`
        realline: distribuciones continuas en el dominio (-inf, +inf)
        realplus: distribuciones continuas en el dominio [0, +inf)
        real0to1: distribuciones continuas en el dominio [0,1]
        discreta: distribuciones discretas
    
    ordenar : {'aic', 'bic'}
        criterio de ordenación de mejor a peor ajuste.
    
    verbose : bool
        Si se muestra información de las distribuciones seleccionadas
        (the default `True`).
        
    Returns
    -------
    resultados: data.frame
        distribucion: nombre de la distribución.
        log_likelihood: logaritmo del likelihood del ajuste.
        aic: métrica AIC.
        bic: métrica BIC.
        n_parametros: número de parámetros de la distribución de la distribución.
        parametros: parámetros del tras el ajuste
        
    Raises
    ------
    Exception
        Si `familia` es distinto de 'realall', 'realline', 'realplus', 'real0to1',
        o 'discreta'.
        
    Notes
    -----

    '''
    
    distribuciones = seleccionar_distribuciones(familia=familia, verbose=verbose)
    distribucion_ = []
    log_likelihood_= []
    aic_ = []
    bic_ = []
    n_parametros_ = []
    parametros_ = []
    
    for i, distribucion in enumerate(distribuciones):
        
        # print(f"{i+1}/{len(distribuciones)} Ajustando distribución: {distribucion.name}")
        
        try:
            parametros = distribucion.fit(data=x)
            nombre_parametros = [p for p in inspect.signature(distribucion._pdf).parameters \
                                 if not p=='x'] + ["loc","scale"]
            parametros_dict = dict(zip(nombre_parametros, parametros))
            log_likelihood = distribucion.logpdf(x, *parametros).sum()
            aic = -2 * log_likelihood + 2 * len(parametros)
            bic = -2 * log_likelihood + np.log(x.shape[0]) * len(parametros)
            
            distribucion_.append(distribucion.name)
            log_likelihood_.append(log_likelihood)
            aic_.append(aic)
            bic_.append(bic)
            n_parametros_.append(len(parametros))
            parametros_.append(parametros_dict)
            
            resultados = pd.DataFrame({
                            'distribucion': distribucion_,
                            'log_likelihood': log_likelihood_,
                            'aic': aic_,
                            'bic': bic_,
                            'n_parametros': n_parametros_,
                            'parametros': parametros_,
                
                         })
            
            resultados = resultados.sort_values(by=ordenar).reset_index(drop=True)
            
        except Exception as e:
            print(f"Error al tratar de ajustar la distribución {distribucion.name}")
            print(e)
            print("")
            
    return resultados

def plot_distribucion(x, nombre_distribucion, ax=None):
    '''
    Esta función superpone la curva de densidad de una distribución con el
    histograma de los datos.
    
    Parameters
    ----------
    x : array_like
        datos con los que ajustar la distribución.
        
    nombre_distribuciones : str
        nombre de una de las distribuciones disponibles en `scipy.stats`.
        
    Returns
    -------
    resultados: matplotlib.ax
        gráfico creado
        
    Raises
    ------
        
    Notes
    -----
    '''

    distribucion = getattr(stats, nombre_distribucion)

    parametros = distribucion.fit(data=x)

    nombre_parametros = [p for p in inspect.signature(distribucion._pdf).parameters \
                         if not p=='x'] + ["loc","scale"]
    parametros_dict = dict(zip(nombre_parametros, parametros))

    log_likelihood = distribucion.logpdf(x, *parametros).sum()

    aic = -2 * log_likelihood + 2 * len(parametros)
    bic = -2 * log_likelihood + np.log(x.shape[0]) * len(parametros)

    x_hat = np.linspace(min(x), max(x), num=100)
    y_hat = distribucion.pdf(x_hat, *parametros)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(7,4))
        
    ax.plot(x_hat, y_hat, linewidth=2, label=distribucion.name)
    ax.hist(x=x, density=True, bins=30, color="#3182bd", alpha=0.5,edgecolor='k');
    ax.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
    ax.set_title('Ajuste distribución')
    ax.set_xlabel('x')
    ax.set_ylabel('Densidad de probabilidad')
    ax.legend();

    print('---------------------')
    print('Resultados del ajuste')
    print('---------------------')
    print(f"Distribución:   {distribucion.name}")
    print(f"Dominio:        {[distribucion.a, distribucion.b]}")
    print(f"Parámetros:     {parametros_dict}")
    print(f"Log likelihood: {log_likelihood}")
    print(f"AIC:            {aic}")
    print(f"BIC:            {bic}")
    
    return ax

def plot_multiple_distribuciones(x, nombre_distribuciones, ax=None):
    '''
    Esta función superpone las curvas de densidad de varias distribuciones
    con el histograma de los datos.
    
    Parameters
    ----------
    x : array_like
        datos con los que ajustar la distribución.
        
    nombre_distribuciones : list
        lista con nombres de distribuciones disponibles en `scipy.stats`.
        
    Returns
    -------
    resultados: matplotlib.ax
        gráfico creado
        
    Raises
    ------
        
    Notes
    -----
    '''
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(7,4))
        
    ax.hist(x=x, density=True, bins=30, color="#3182bd", alpha=0.5,edgecolor='k')
    ax.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
    ax.set_title('Ajuste distribuciones')
    ax.set_xlabel('x')
    ax.set_ylabel('Densidad de probabilidad')
    
    for nombre in nombre_distribuciones:
        
        distribucion = getattr(stats, nombre)

        parametros = distribucion.fit(data=x)

        nombre_parametros = [p for p in inspect.signature(distribucion._pdf).parameters \
                             if not p=='x'] + ["loc","scale"]
        parametros_dict = dict(zip(nombre_parametros, parametros))

        log_likelihood = distribucion.logpdf(x, *parametros).sum()

        aic = -2 * log_likelihood + 2 * len(parametros)
        bic = -2 * log_likelihood + np.log(x.shape[0]) * len(parametros)

        x_hat = np.linspace(min(x), max(x), num=100)
        y_hat = distribucion.pdf(x_hat, *parametros)
        ax.plot(x_hat, y_hat, linewidth=2, label=distribucion.name)
    
    ax.legend();
    
    return ax

def EOS1(P, T , MW=24): 
  """
  Fluid Data
  P  : Pressure ,psi
  T  : Temperature, °F
  Gas Properties
  MW  = 24  Molecular Weight, MW
  """
  SGg = MW/28.966 
  Ppc = 677 + 15*SGg - 37.5*(SGg**2)
  Tpc = 168 + 325*SGg - 12.5*(SGg**2)  
  Tpr = (T+459.9)/Tpc
  Ppr = (P+14.7)/Ppc
  A = 1.39*((Tpr - 0.92)**(0.5)) - 0.36*Tpr - 0.1
  B = (0.62 - 0.23*Tpr)*Ppr + ((0.066/(Tpr - 0.86)) - 0.037)*Ppr**2 + 0.32*(Ppr**6)/(10**(9*(Tpr - 1)))
  C = 0.132 - 0.32*m.log10(Tpr)
  D = 10**(0.3106 - 0.49*Tpr + 0.1824*Tpr**2)
  z = A + (1 - A)/m.exp(B) + C*Ppr**D 
  return z

def FlowGasCal(P1, P2, Tf, zf, IDt, Lt, FactD= 1.000, ε=0.0000709, MW=24, μ=0.0103, k=1.41, Pb=14.7, Tb=60, zb=1.000):
  """
  Pipe Data			
  IDt:  Pipe Size, in
  Lt :  Length, ft
  Fluid Data			
  P1 :  Inlet Pressure, psi
  P2 :  Outlet Pressure, psi
  Tf :  Flowing Temperature, °F
  zf :  Compressibility Factor, adim
  
  ### Pipe Properties 
  ε     = 0.0000709  Pipe Roughness, in
  FactD = 1.000	     Factor D of pipe

  ### Fluid Properties 
  MW = 24            Molecular Weight, MW
  μ  = 0.0103        Viscosity, cp
  k  = 1.4           Specific Heat Ratio, Cp/Cv
  Pb = 14.7          Basic Pressure at standard conditions
  Tb = 60            Basic Temperature at standard conditions
  zb = 1             Basic Compresibility factor at standard conditions
  """
  rhof = ((P1*MW)/((Tf+459.67)*zf*10.7316))              # Fluid Density @ (P1, Tf) lb/ft3
  IDt  = IDt*FactD                                       # Pipe Inside Diameter, in
  At   = m.pi*(IDt/24)**2                                # A (Area), ft2

  flow_rate_fluid = 0.5                      # lb/s
  for i in range(100):
    volumetric_flow = flow_rate_fluid/rhof   # ft3/s
    velocity        = volumetric_flow/At     # ft/s
    reynols_number  = (IDt/12)*velocity*rhof/(μ/1488.1639)

    if reynols_number<=2100:
      f = 64/reynols_number
      flow_rate_fluid_cal = m.sqrt(((144*32.174*(At**2)*rhof)/(f*(Lt/(IDt/12))+2*m.log(P1/P2)))*((P1**2-P2**2)/P1))
      flow_rate_fluid = flow_rate_fluid_cal
    else:
      f = 64/reynols_number
      for i in range(100):
        fcal = 1/((-2*m.log10((((ε/IDt)/3.7)+(2.51/(reynols_number*m.sqrt(f))))))**2)
        f    = fcal
    flow_rate_fluid_cal = m.sqrt(((144*32.174*(At**2)*rhof)/(f*(Lt/(IDt/12))+2*m.log(P1/P2)))*((P1**2-P2**2)/P1))
    flow_rate_fluid = flow_rate_fluid_cal
  volumetric_flowrate = flow_rate_fluid*60/rhof
  QgSCFM = (volumetric_flowrate*P1/(Tf+459.67))*(60+459.67)/14.7                                         
  QgSCFD = QgSCFM*60*24/1E+6
  return QgSCFD

def FlowGas_CheckTuned(P1,P2,Tf,IDt,Lt):
  if (P1<0)|(P2<0):
    Qgsc = 0
    s    = 'Not Flow'
  elif P1>P2:
    zf   = EOS1(P1, Tf)
    Qgsc = FlowGasCal(P1, P2, Tf, zf, IDt, Lt)
    s    = 'Dir Flow'
  elif P1==P2:
    Qgsc = 0
    s    = 'Not Flow'
  else:
    zf   = EOS1(P2, Tf)
    Qgsc = FlowGasCal(P2, P1, Tf, zf, IDt, Lt)
    s    = 'Rev Flow'
  return [round(Qgsc,3), s]

def InletOutletGasFlow(Qgsc):
  if Qgsc==0:
    Qg = 0
    s  = 'Not Flow'
  elif Qgsc<0:
    Qg = -Qgsc
    s = 'Outlet Flow'
  elif Qgsc>0: 
    Qg = +Qgsc
    s  = 'Inlet Flow'
  else :
    Qg = np.nan 
    s  = np.nan 
  return [Qg,s]

In [6]:
# disttribution and box ploting
def graphing_dist_box_plot(s,lw_lim,up_lim,limits=True,dist='norm',P1=25,P2=50,P3=75,
                           color='blue',figsize=(4,4),xlabel='Variable',ylabel='Probability Density',title='Distribution Curve'):
  x = s
  if limits:
    lw_lim=lw_lim
    up_lim=up_lim
  else:
    lw_lim=min(x)
    up_lim=max(x)
  
  x = x[x>lw_lim]
  x = x[x<up_lim]
  P_1=np.percentile(x, P1) 
  P_2=np.percentile(x, P2)  
  P_3=np.percentile(x, P3)
  f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.25, 1)},figsize=figsize)
  color = color
  title = title
  xlabel = xlabel
  ylabel = ylabel
  sns.boxplot(x, ax=ax_box, color=color)
  ################ InPut distribution
  distx =  getattr(stats,dist)
  sns.distplot(x, ax=ax_hist, fit=distx, kde=False, color=color,hist_kws={'edgecolor':'k'})
  ax_box.axvline(P_1,  color=color, linestyle='--')
  ax_box.axvline(P_2,  color=color, linestyle='-')
  ax_box.axvline(P_3,  color=color, linestyle='--')
  ax_hist.axvline(P_1, color=color, linestyle='--')
  ax_hist.axvline(P_2, color=color, linestyle='-')
  ax_hist.axvline(P_3, color=color, linestyle='--')
  plt.legend({distx.name:distx,str(P1)+'%':P_1,str(P2)+'%':P_2,str(P3)+'%':P_3})
  ax_box.set(xlabel='')
  ax_box.set_title(title)
  ax_hist.set_xlabel(xlabel), ax_hist.set_ylabel(ylabel)
  plt.show()